# Model with Feedback – Logistic vs XGB vs NN
This notebook is part of a multi-step project designed to model customer churn using both structured features and GPT-generated customer feedback.
We explore how customer feedback can improve churn prediction performance.


### Project Structure:

1. **Exploring Customer Churn & GPT-generated Feedback**  
   → Data exploration & text insights

2. **Feedback Noise Simulation & Fallback Testing**  
   → Simulate real-world feedback imperfections

3. **Fallback Model – Logistic vs XGB vs RF vs NN**  
   → Structured-only models, compared & evaluated

4. **Model with Feedback – Logistic vs XGB vs NN**  
   → Hybrid modeling using structured + textual data

5. **Prediction using Synthetic Feedback Knowledge**  
   → Transfer feedback model knowledge to fallback model



## 📑 Table of Contents

1. [Imports & config](#1-imports--config)
2. [Load data](#2-load-data)
3. [Features config](#3-features-config)
4. [Preprocessing pipeline](#4-preprocessing-pipeline)
5. [Train models](#5-train-models)
6. [Summary](#6-summary) <br>
[📁 Notebook Series Overview: Telco Churn & GPT Feedback](#notebook-series-overview-telco-churn--gpt-feedback)


## 1. Imports & config
<a id="1-imports--config"></a>


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV


from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from textblob import TextBlob

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
pd.set_option('display.max_colwidth', None)

## 2. Load data
<a id="2-load-data"></a>


In [3]:
df = pd.read_csv('/kaggle/input/telco-customer-churn-realistic-customer-feedback/telco_noisy_feedback_prep.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,...,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,CustomerFeedback,feedback_length,sentiment,HasFeedback
0,0,0,7590-vhveg,female,0,yes,no,1,no,no phone service,...,month-to-month,yes,electronic check,29.85,29.85,0,NaN,0,0.0,False
1,1,1,5575-gnvde,male,0,no,no,34,yes,no,...,one year,no,mailed check,56.95,1889.50,0,NaN,0,0.0,False
2,2,2,3668-qpybk,male,0,no,no,2,yes,no,...,month-to-month,yes,mailed check,53.85,108.15,1,NaN,0,0.0,False
3,3,3,7795-cfocw,male,0,no,no,45,no,no phone service,...,one year,no,bank transfer (automatic),42.30,1840.75,0,NaN,0,0.0,False
4,4,4,9237-hqitu,female,0,no,no,2,yes,no,...,month-to-month,yes,electronic check,70.70,151.65,1,NaN,0,0.0,False


## 3. Features config
<a id="3-features-config"></a>


In [4]:
# ==== Feature config ====
df_text = df[df["HasFeedback"]].copy()

# Target and feature setup
target_col = "Churn"
text_col = "CustomerFeedback"
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'feedback_length', 'sentiment']
categorical_cols = ["Contract", "PaymentMethod", "InternetService", 
    "Partner", "Dependents"]

# Define features and target
X_text = df_text[numeric_cols + categorical_cols + [text_col]]
y_text = df_text[target_col]

# Stratified split to preserve churn distribution
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_text, y_text, test_size=0.2, random_state=42, stratify=y_text
)


## 4. Preprocessing pipeline
<a id="4-preprocessing-pipeline"></a>


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Define transformers
num_transformer = Pipeline([('scaler', StandardScaler())])
cat_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])
txt_transformer = Pipeline([('tfidf', TfidfVectorizer(max_features=300))])

# Combine all into one ColumnTransformer
preprocessor_text = ColumnTransformer([
    ('num', num_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols),
    ('txt', txt_transformer, text_col)
])

## 5. Train models
<a id="5-train-models"></a>


In [6]:
# Define models to compare
models_with_text = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "Neural Net (MLP)": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
}

# Store results
results_with_text = {}

# Train and evaluate
for name, model in models_with_text.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor_text),
        ("classifier", model)
    ])
    pipe.fit(X_train_text, y_train_text)
    y_pred = pipe.predict(X_test_text)
    y_prob = pipe.predict_proba(X_test_text)[:, 1]

    report = classification_report(y_test_text, y_pred, output_dict=True)
    auc = roc_auc_score(y_test_text, y_prob)

    results_with_text[name] = {
        "f1-score": report["weighted avg"]["f1-score"],
        "accuracy": report["accuracy"],
        "roc_auc": auc
    }

# Show results as a table
results_text_df = pd.DataFrame(results_with_text).T.sort_values("roc_auc", ascending=False)
display(results_text_df)

,f1-score,accuracy,roc_auc
Neural Net (MLP),0.997164,0.997159,0.999959
Logistic Regression,0.997154,0.997159,0.999918
XGBoost,0.994299,0.994318,0.998722


All models perform similarly – Logistic Regression wins in simplicity, interpretability, and stability.



In [7]:
# Build and train best model explicitly
best_pipe = Pipeline([
    ("preprocessor", preprocessor_text),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

best_pipe.fit(X_train_text, y_train_text)

# Save
import joblib
joblib.dump(best_pipe, "model_with_feedback.pkl")
print("Logistic Regression with text saved.")

Logistic Regression with text saved.


## Summary
<a id="6-summary"></a>


In this notebook, we trained multiple models using both structured features and customer feedback.

### Key findings:
- All models performed exceptionally well, thanks to the predictive power of textual feedback.
- **Logistic Regression** delivered the best overall performance (ROC AUC = 0.9999), with a perfect balance between accuracy and recall.
- We saved this model as `model_with_feedback.pkl`.

In next notebook we will **transfer knowledge** from this feedback-based model to improve predictions in cases where feedback is missing.  
This will be done by introducing a synthetic variable: `synthetic_text_score` derived from the trained model.

## 📁 Notebook Series Overview: **Telco Churn & GPT Feedback**
<a id="notebook-series-overview-telco-churn--gpt-feedback"></a>

This is part of a multi-notebook series focused on **realistic churn prediction** using structured data and GPT-generated customer feedback.  
We simulate real-world challenges such as missing text, noisy input, and model fallback logic — building toward a robust, production-style solution.

---

###  1. Exploring Customer Churn & GPT-generated Feedback

> Exploratory analysis of the churn dataset, with focus on customer segments, behavior patterns, and text feedback characteristics.  
> Text insights powered by GPT-generated responses.

---

###  2. Feedback Noise Simulation & Fallback Testing

> We simulate feedback imperfections (irrelevant, incomplete, or missing entries) and measure how models perform under noise.  
> Introduces the concept of fallback systems for low-signal situations.

---

### 3. Fallback Model – Logistic vs XGB vs RF vs NN

> We train and compare multiple classifiers on **structured-only data** (no feedback):  
> Logistic Regression, Random Forest, XGBoost, and Neural Net.  
> Best fallback model is selected based on ROC AUC and recall.

---

###  4. Model with Feedback – Logistic vs XGB vs NN

> We focus on customers who left written feedback.  
> Structured data is combined with TF-IDF vectors to train a stronger “teacher” model.  
> Logistic Regression emerges as the most interpretable and best-performing solution.

---

### 5. Prediction using Synthetic Feedback Knowledge

> Even when customers don’t leave feedback, we simulate its predictive signal using the trained teacher model.  
> We generate a `synthetic_text_score` and evaluate whether it helps fallback performance.

---
